In [7]:
import re

# Ruta al archivo original
entrada = "inspecciones20251.txt"
salida = "inspecciones2025_filtrado.txt"

bloques_validos = []
bloque_actual = []

# Patrón que detecta el inicio de un nuevo mensaje con fecha y hora
patron_fecha = re.compile(r"^\d{1,2}/\d{1,2}/\d{4}, \d{1,2}:\d{2}")

with open(entrada, "r", encoding="utf-8") as file:
    for linea in file:
        # Si encontramos una nueva fecha (nuevo mensaje)
        if patron_fecha.match(linea):
            # Revisamos si el bloque anterior tenía "O.D.T" o "Orden de trabajo"
            if bloque_actual:
                texto_completo = " ".join(bloque_actual).lower()
                if "o.d.t" in texto_completo or "orden de trabajo" in texto_completo:
                    bloques_validos.append("\n".join(bloque_actual))
            bloque_actual = [linea.strip()]
        else:
            bloque_actual.append(linea.strip())

# Verificamos el último bloque
if bloque_actual:
    texto_completo = " ".join(bloque_actual).lower()
    if "o.d.t" in texto_completo or "orden de trabajo" in texto_completo:
        bloques_validos.append("\n".join(bloque_actual))

# Guardamos solo los mensajes válidos
with open(salida, "w", encoding="utf-8") as out:
    out.write("\n\n".join(bloques_validos))

print("✅ Archivo filtrado generado: inspecciones2025_filtrado.txt")



✅ Archivo filtrado generado: inspecciones2025_filtrado.txt


In [16]:
import re
import pandas as pd
import unicodedata

def limpiar(texto):
    """Convierte texto a minúsculas, sin tildes, y sin espacios invisibles."""
    texto = texto.lower().strip()
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))
    return texto

# Archivo de entrada ya filtrado
archivo = "inspecciones2025_filtrado.txt"

# Leer contenido completo
with open(archivo, "r", encoding="utf-8") as file:
    contenido = file.read()

# Dividir en bloques que empiezan con fecha
bloques = re.split(r"(?=\d{1,2}/\d{1,2}/\d{4}, \d{1,2}:\d{2})", contenido)

# Lista de registros finales
registros = []

for bloque in bloques:
    if not bloque.strip():
        continue

    # Limpiar y dividir líneas del bloque
    lineas = bloque.strip().splitlines()
    lineas = [limpiar(l) for l in lineas]

    # Detectar fecha
    match_fecha = re.match(r"^(\d{1,2}/\d{1,2}/\d{4}, \d{1,2}:\d{2})", lineas[0])
    fecha_actual = match_fecha.group(1) if match_fecha else ""

    # Detectar posiciones de O.D.T
    indices_odt = [i for i, l in enumerate(lineas) if "o.d.t" in l]

    for i, idx in enumerate(indices_odt):
        # Extraer número de ODT
        odt_match = re.search(r"o\.d\.t\s*(\d+)", lineas[idx])
        odt = odt_match.group(1) if odt_match else ""
        fin = indices_odt[i + 1] if i + 1 < len(indices_odt) else len(lineas)


        categoria = ""
        inspectores = ""
        direccion = ""

        # Buscar datos desde esa ODT hasta la próxima
        for j in range(idx + 1, fin):
            l = lineas[j]

            # Categoría flexible
            if not categoria and ("cat" in l or "reinspeccion" in l or "denuncia" in l or "integral" in l):
                categoria = l.strip()

            # Inspectores
            elif not inspectores and "insp." in l:
                inspectores = l.replace("insp.", "").strip()

            # Dirección (con número o esquina con cardinal)
            elif not direccion and (
                re.search(r"\d", l) or
                re.search(r"esq.*(n|s)[/\.]?(e|o)", l) or
                "esq." in l 
            ):
                direccion = l.strip()
                break  # solo tomamos la primera dirección

        # Guardar registro
        registros.append({
            "Fecha": fecha_actual,
            "ODT": odt,
            "Categoría": categoria,
            "Inspectores": inspectores,
            "Dirección": direccion
        })

# Crear DataFrame
df = pd.DataFrame(registros)

# Exportar resultados
df.to_csv("odt_inspecciones_final.csv", index=False)
df.to_excel("odt_inspecciones_final.xlsx", index=False)

print(f"\n✅ Listo Luis. Se exportaron {len(df)} O.D.T con dirección, categoría e inspectores.")



✅ Listo Luis. Se exportaron 920 O.D.T con dirección, categoría e inspectores.
